<a href="https://colab.research.google.com/github/Manuel-Tellechea/Financial_trading_with_Python/blob/main/074_Extract_Economic_US_FED_Data_using_Python_for_Real_Estate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract Economic US Federal Reserve Economic Data

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com                                                      |
| External References   | Pandas Datareader & Federal Reserve Economic Data (FRED) |
| Input Datasets        | FRED API doc key                                                                                    |
| Output Datasets       | Series values for time range |
| Input Data Source     | String |
| Output Data Source    | CSV |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 26th Sep 2021 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Request [FRED API Key](https://fred.stlouisfed.org/docs/api/api_key.html)

## Useful Resources
- [Google Collab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)
- [NLP Resource](https://stackabuse.com/python-for-nlp-parts-of-speech-tagging-and-named-entity-recognition/)
- [Pandas Datareader & Federal Reserve Economic Data (FRED)](https://medium.com/swlh/pandas-datareader-federal-reserve-economic-data-fred-a360c5795013)
- [Predicting The Housing Market Is Easier Than You Think](https://medium.com/swlh/predicting-the-housing-market-is-easier-than-you-think-45239a366dc1)
- [Plotly Express](https://plotly.com/python/basic-charts/)

## <font color="blue">Install Packages</font>

In [ ]:
!pip install pandas-datareader -q

## <font color="blue">Imports</font>

In [ ]:
from google.colab import drive, files # specific to Google Colab
import pandas_datareader as pdr # access fred
import pandas as pd
import requests # data from api
import plotly.express as px # visualize
from datetime import datetime

## <font color="blue">Functions</font>

In [ ]:
def get_fred_series_data(api_key,
                         series):
  # url
  url = "https://api.stlouisfed.org/geofred/series/data?series_id={0}&api_key={1}&file_type=json".format(series, api_key)
  # response
  response = requests.request("GET", url)
  return response

In [ ]:
def transform_series_response(response):
  latest_date = list(response.json()['meta']['data'].keys())[0]
  return pd.DataFrame(response.json()['meta']['data'][latest_date])

In [ ]:
def get_fred_data(param_list, start_date, end_date):
  df = pdr.DataReader(param_list, 'fred', start_date, end_date)
  return df.reset_index()

## <font color="blue">Locals & Constants</font>

In [ ]:
############
# OPTIONAL #
############

# mount drive
drive.mount('/content/drive', force_remount=False)

# data location
file_dir = '/content/drive/My Drive/Colab Data/input/' # optional

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + 'api_keys.csv') # if you have your key stored in a csv file

# get keys
fred_api_key = df_api_keys.loc[df_api_keys['API'] =='fred']['KEY'].iloc[0] # replace this with your own key

## <font color="blue">Data</font>

In [ ]:
series = 'CABPPRIVSA' # https://fred.stlouisfed.org/series/CABPPRIVSA

In [ ]:
# get data for series
df = get_fred_data(param_list=['CABPPRIVSA'], 
                   start_date='2021-01-01', 
                   end_date='2021-08-31')
df

,DATE,CABPPRIVSA
0,2021-01-01,10390.113831
1,2021-02-01,9965.519798
2,2021-03-01,11209.312182
3,2021-04-01,10416.523695
4,2021-05-01,9449.807853
5,2021-06-01,9731.414489
6,2021-07-01,9716.201466
7,2021-08-01,10197.320338


In [ ]:
# plot
fig = px.line(df, x="DATE", y="CABPPRIVSA", title='New Private Housing Units Authorized by Building Permits')
fig.show()

In [ ]:
# get all series ids per series
response = get_fred_series_data(fred_api_key, series)
# transform response into a dataframe
df_all_series_ids = transform_series_response(response)
df_all_series_ids.head()

,region,code,value,series_id
0,Alabama,01,1593.2260332958201,ALBPPRIVSA
1,Alaska,02,122.86647951583599,AKBPPRIVSA
2,Arizona,04,5526.523667435191,AZBPPRIVSA
3,Arkansas,05,1219.25749966006,ARBPPRIVSA
4,California,06,9072.025973455658,CABPPRIVSA


In [ ]:
# get all series to a list
series_list = df_all_series_ids['series_id'].tolist()
print('Length of series list:', len(series_list) + 1)
series_list[:5] # show first five in list

Length of series list: 52


['ALBPPRIVSA', 'AKBPPRIVSA', 'AZBPPRIVSA', 'ARBPPRIVSA', 'CABPPRIVSA']

In [ ]:
# set range for time
start_date = '2021-01-01'
end_date = datetime.today().strftime('%Y-%m-%d') # today

# get series data
df_permits_all_series = get_fred_data(param_list=series_list, # all series to get data for
                                      start_date=start_date, # start date
                                      end_date=end_date) # get latest date
df_permits_all_series.head()

,DATE,ALBPPRIVSA,AKBPPRIVSA,AZBPPRIVSA,ARBPPRIVSA,CABPPRIVSA,COBPPRIVSA,CTBPPRIVSA,DEBPPRIVSA,DCBPPRIVSA,FLBPPRIVSA,GABPPRIVSA,HIBPPRIVSA,IDBPPRIVSA,ILBPPRIVSA,INBPPRIVSA,IABPPRIVSA,KSBPPRIVSA,KYBPPRIVSA,LABPPRIVSA,MEBPPRIVSA,MDBPPRIVSA,MABPPRIVSA,MIBPPRIVSA,MNBPPRIVSA,MSBPPRIVSA,MOBPPRIVSA,MTBPPRIVSA,NEBPPRIVSA,NVBPPRIVSA,NHBPPRIVSA,NJBPPRIVSA,NMBPPRIVSA,NYBPPRIVSA,NCBPPRIVSA,NDBPPRIVSA,OHBPPRIVSA,OKBPPRIVSA,ORBPPRIVSA,PABPPRIVSA,RIBPPRIVSA,SCBPPRIVSA,SDBPPRIVSA,TNBPPRIVSA,TXBPPRIVSA,UTBPPRIVSA,VTBPPRIVSA,VABPPRIVSA,WABPPRIVSA,WVBPPRIVSA,WIBPPRIVSA,WYBPPRIVSA
0,2021-01-01,1921.453718,152.806785,5610.508075,1314.522152,10390.113831,5173.979915,366.476383,730.660505,313.0,15677.524822,5396.923001,426.698260,1770.241081,1566.996619,2366.473459,1011.602521,704.316872,1256.424884,1647.990128,706.789255,1395.661599,1419.828180,2338.257967,3088.585949,720.626307,1434.668928,402.221211,861.294002,2182.656928,299.987564,3603.434898,480.839846,3331.860275,6730.227504,1027.252368,2356.761595,1362.609162,1392.192307,4567.617250,129.233300,3796.427094,1076.397031,4381.189352,22189.778420,3366.081907,169.612592,3310.890019,4896.936343,250.881034,1924.417489,268.443200
1,2021-02-01,1957.649706,155.365440,5896.913415,1165.607212,9965.519798,4354.655218,361.288587,807.859415,749.0,13964.868341,4818.709553,344.057485,1651.084185,1525.460572,2250.236074,1268.072915,1068.545063,1886.768219,1545.696780,434.090514,2189.076724,1478.364682,2039.258701,3025.734667,682.513868,1573.964183,381.517143,781.992464,2057.866022,379.673456,2809.354849,725.528050,3906.952803,7780.129445,336.484238,2751.997301,1199.349634,1685.093735,3605.824787,123.361018,3895.055762,512.552384,4247.497785,19997.879200,3473.814988,144.269616,2962.897993,5538.859554,766.809713,2076.849616,200.514194
2,2021-03-01,1894.970984,162.989892,5128.552754,1725.865817,11209.312182,4774.678537,456.113837,804.991634,589.0,17011.884896,5620.425940,261.065591,1899.731558,1454.249201,2411.704851,1800.156429,809.074260,1598.443794,1695.505833,543.682570,1450.660100,1532.155281,2287.818345,3304.295097,804.296686,1851.955403,634.171049,939.817513,2105.593119,443.476335,2952.662717,618.317613,3372.323282,8020.899998,432.817679,2693.682545,1361.456315,1999.220329,3635.463496,135.433434,3989.785968,711.611025,5508.113233,21842.221592,3504.707762,158.587718,4037.228352,4537.900335,396.489515,2160.033374,202.984733
3,2021-04-01,1546.695705,154.461556,5509.318073,1211.519195,10416.523695,5420.311199,697.646593,821.262045,159.0,17961.652293,5840.728930,358.377552,1724.896279,1739.716505,2249.366491,1761.737547,769.684233,1240.411508,2290.392452,587.179160,1431.917744,2246.120573,1960.930871,3041.349456,734.465300,2167.074937,407.220001,838.611498,2153.638872,423.184910,3958.885389,675.487339,3190.209887,7967.924319,250.802116,2721.269408,1358.705732,1933.106907,2556.480688,157.782041,4284.234296,687.807087,5108.948539,23231.185959,3884.112389,225.286468,3635.406361,4038.079991,251.308559,2124.819597,197.991718
4,2021-05-01,1707.017867,121.455814,5158.607123,941.540904,9449.807853,4650.795093,314.162382,813.949636,403.0,18112.800204,5266.436167,231.568706,1368.245968,1452.253941,2033.916319,848.035410,672.402773,1170.528588,1599.651083,599.910608,1227.048460,1604.005679,1776.218298,3242.937646,802.854704,1463.525884,332.137580,860.947686,1574.829333,499.328791,3054.783531,561.933155,3982.765404,6672.729216,303.851771,2746.145030,1131.597914,1848.683792,2543.521639,113.135660,3915.056269,571.498902,5051.696873,21092.114002,3291.470167,168.736675,3002.853041,4026.601650,287.116523,1930.771096,148.219274


In [ ]:
# transform columns to single column
df_melt = pd.melt(df_permits_all_series, id_vars=['DATE'], value_vars=series_list, var_name='STATE', value_name='PERMITS')
df_melt.head()

,DATE,STATE,PERMITS
0,2021-01-01,ALBPPRIVSA,1921.453718
1,2021-02-01,ALBPPRIVSA,1957.649706
2,2021-03-01,ALBPPRIVSA,1894.970984
3,2021-04-01,ALBPPRIVSA,1546.695705
4,2021-05-01,ALBPPRIVSA,1707.017867


In [ ]:
# modify state abbreviation
df_plot = df_melt.copy() # copy df
df_plot['STATE'] = df_plot.apply(lambda x: x['STATE'][:2], axis=1)
df_plot.head()

,DATE,STATE,PERMITS
0,2021-01-01,AL,1921.453718
1,2021-02-01,AL,1957.649706
2,2021-03-01,AL,1894.970984
3,2021-04-01,AL,1546.695705
4,2021-05-01,AL,1707.017867


In [ ]:
# plot
fig = px.line(df_plot, 
              x="DATE", # horizontal axis
              y="PERMITS", # vertical axis
              color='STATE', # split column
              title='New Private Housing Units Authorized by Building Permits')
fig.show()

In [ ]:
# download file
file_name = f'{series}_{start_date}-{end_date}.csv'
df_plot.to_csv(file_name, index=False)
files.download(file_name)
print('Download {0}'.format(file_name))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download CABPPRIVSA_2021-01-01-2022-02-17.csv


# End Notebook